## Set Environmental Variable

In [32]:
import nest_asyncio
from langchain_community.chains.graph_qa.gremlin import GremlinQAChain
from langchain_community.graphs import GremlinGraph
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from openai import AzureOpenAI

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

azure_cosmodb_gremlin_endpoint = os.getenv("AZURE_COSMODB_GREMLIN_ENDPOINT")
azure_cosmodb_gremlin_username = os.getenv("AZURE_COSMODB_GREMLIN_USERNAME")
azure_cosmodb_gremlin_key = os.getenv("AZURE_COSMODB_GREMLIN_KEY")


## Load the Data

In [3]:
import pandas as pd
df = pd.read_csv('data/calories.csv')
df.head()

,FoodCategory,FoodItem,per100grams,Cals_per100grams,KJ_per100grams
0,CannedFruit,Applesauce,100g,62 cal,260 kJ
1,CannedFruit,Canned Apricots,100g,48 cal,202 kJ
2,CannedFruit,Canned Blackberries,100g,92 cal,386 kJ
3,CannedFruit,Canned Blueberries,100g,88 cal,370 kJ
4,CannedFruit,Canned Cherries,100g,54 cal,227 kJ


In [4]:
df.isnull().sum()

FoodCategory        0
FoodItem            0
per100grams         0
Cals_per100grams    0
KJ_per100grams      0
dtype: int64

In [5]:
foodcategory_list = df['FoodCategory'].unique()
foodcategory_list

array(['CannedFruit', 'Fruits', 'Tropical&ExoticFruits', 'PotatoProducts',
       'Vegetables', 'FastFood', 'Pizza', 'Cheese', 'CreamCheese',
       'Milk&DairyProducts', 'SlicedCheese', 'Yogurt', 'Beef&Veal',
       'ColdCuts&LunchMeat', 'Meat', 'Offal&Giblets', 'Pork',
       'Poultry&Fowl', 'Sausage', 'Venison&Game', 'Cakes&Pies',
       'Candy&Sweets', 'IceCream', '(Fruit)Juices',
       'AlcoholicDrinks&Beverages', 'Beer',
       'Non-AlcoholicDrinks&Beverages', 'Soda&SoftDrinks', 'Wine',
       'CerealProducts', 'Oatmeal,Muesli&Cereals', 'Pasta&Noodles',
       'Dishes&Meals', 'Soups', 'Legumes', 'Nuts&Seeds', 'Oils&Fats',
       'VegetableOils', 'BakingIngredients', 'Fish&Seafood',
       'Herbs&Spices', 'Pastries,Breads&Rolls', 'Sauces&Dressings',
       'Spreads'], dtype=object)

In [6]:
df.columns

Index(['FoodCategory', 'FoodItem', 'per100grams', 'Cals_per100grams',
       'KJ_per100grams'],
      dtype='object')

In [63]:
df[df['FoodCategory']=="Fruits"]

,FoodCategory,FoodItem,per100grams,Cals_per100grams,KJ_per100grams
29,Fruits,Acai,100g,70 cal,294 kJ
30,Fruits,Apple,100g,52 cal,218 kJ
31,Fruits,Applesauce,100g,68 cal,286 kJ
32,Fruits,Apricot,100g,48 cal,202 kJ
33,Fruits,Avocado,100g,160 cal,672 kJ
34,Fruits,Banana,100g,89 cal,374 kJ
35,Fruits,Blackberries,100g,43 cal,181 kJ
36,Fruits,Blood Oranges,100g,50 cal,210 kJ
37,Fruits,Blueberries,100g,57 cal,239 kJ
38,Fruits,Cantaloupe,100g,34 cal,143 kJ


## Build Graph and Upload the graph into Azure CosmoDB(Gremlin)

In [8]:
graph = GremlinGraph(
    url=azure_cosmodb_gremlin_endpoint,
    username=azure_cosmodb_gremlin_username,
    password=azure_cosmodb_gremlin_key
)

In [9]:
foodcategory_list =['Vegetables','FastFood','Fish&Seafood','Meat','Fruits']

In [10]:
for foodcategory in foodcategory_list:
    # Filter the DataFrame for the current category
    df_category = df[df['FoodCategory'] == foodcategory]
    sentence = ""
    node_list = []
    
    # Create initial node for the food category
    initial_node = Node(id=foodcategory, properties={"label": "foodcategory", "title": foodcategory})
    node_list.append(initial_node)
    relationship_list = []
    
    # Iterate over each row in the filtered DataFrame
    for index, row in df_category.iterrows():
        # Create nodes for each food item and its calorie information
        food_node = Node(id=row['FoodItem'], properties={"label": "fooditem", "title": row['FoodItem']})
        node_list.append(food_node)
        cals_contains = Node(id=f"{row['FoodItem']}_Calories", properties={"label": "calories", "title": row['Cals_per100grams']})
        node_list.append(cals_contains)

        # Correct relationships between nodes
        relationship_list.append(Relationship(id=f"{row['FoodItem']}_categorised_as", type="CategorisedAs", source=food_node, target=initial_node, properties={"label": "CategorisedAs"}))
        relationship_list.append(Relationship(id=f"{row['FoodItem']}_calorie_info", type="Calorieper100gm", source=food_node, target=cals_contains, properties={"label": "Calorieper100gm"}))

        # Build the descriptive sentence for the document
        sentence += f" {row['per100grams']} of {row['FoodItem']} in the {row['FoodCategory']} category contains {row['Cals_per100grams']} Calories."

    # Create a source document with the descriptive sentence
    source_doc = Document(page_content=sentence)

    # Create a GraphDocument with nodes and relationships
    graph_doc = GraphDocument(
        nodes=node_list,
        relationships=relationship_list,
        source=source_doc,
    )

    # Apply asyncio workaround
    nest_asyncio.apply()

    # Add the document to the CosmosDB graph
    graph.add_graph_documents([graph_doc])


In [11]:
graph.refresh_schema()

In [12]:
print(graph.schema)

Vertex labels are the following:
foodcategory,fooditem,calories,Calorieper100gm
Edge labes are the following:
CategorisedAs,Calorieper100gm
Vertices have following properties:
{'foodcategory': ['type', 'title'], 'fooditem': ['type', 'title'], 'calories': ['type', 'title'], 'Calorieper100gm': ['type', 'title']}


## Example Implementation

In [13]:
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OAI_MODEL_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OAI_MODEL_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OAI_MODEL_VERSION")

In [61]:
custom_prompt_template = PromptTemplate(
    input_variables=['question', 'schema'],
    template="""Task: Generate a Gremlin query to query a graph database.

Instructions:
- Always use the following format for the Gremlin query:
  g.V().hasLabel('fooditem').has('title', '{item}').outE('Calorieper100gm').inV().values('title')
- Replace {item} with the specific food item being queried.
- Use only the provided relationship types and properties in the schema.
- Do not use any other relationship types or properties that are not provided.

Schema:
Vertices:
- Label: 'fooditem'
  Properties:
    - 'title': string (e.g., 'Beetroot', 'Artichoke', 'Arugula')
    - 'type': string (e.g., 'Node')

Edges:
- Label: 'Calorieper100gm'
  Direction: Outgoing from 'fooditem' vertices
  Connects:
    - From: Vertex labeled 'fooditem' (e.g., 'Beetroot')
    - To: Vertex with calorie information (e.g., title: '43 cal')

For example, if the question is "How many calories are in Beetroot?", the query should be:
g.V().hasLabel('fooditem').has('title', 'Beetroot').outE('Calorieper100gm').inV().values('title')

Must Follow: 
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything other than constructing a Gremlin statement.
Do not include any text except the generated Gremlin statement.
Do not construct anything on your own just follow the example and instruction. 

Construct the Gremlin Query for the following based on Instruction, just give Gremlin Query no need anyother error message or explanation:
{question}
"""
)

chain = GremlinQAChain.from_llm(
    AzureChatOpenAI(
        temperature=0,
        azure_deployment="gpt-35-turbo",
    ),
    graph=graph,
    prompt_template=custom_prompt_template,
    verbose=True,
    allow_dangerous_requests=True
)
